# Import

In [ ]:
import torch

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Example of moving a model and data to GPU
# model = YourModel(...)
# model.to(device)
# data = data.to(device)

Using device: cuda


In [ ]:
import os
import random
import glob
import re

import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler

import torch
import torch.nn as nn
from tqdm import tqdm


# Fixed RandomSeed & Setting Hyperparameter

In [ ]:
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed(42)

In [ ]:
LOOKBACK, PREDICT, BATCH_SIZE, EPOCHS = 28, 7, 16, 50
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Data Load

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train = pd.read_csv('/content/drive/MyDrive/lg_aimers_2/open/train/train.csv')

# Define Model

In [ ]:
class MultiOutputLSTM(nn.Module):
    def __init__(self, input_dim=1, hidden_dim=64, num_layers=2, output_dim=7):
        super(MultiOutputLSTM, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        out, _ = self.lstm(x)
        return self.fc(out[:, -1, :])  # (B, output_dim)

# Train

In [ ]:
def train_lstm(train_df):
    trained_models = {}

    for store_menu, group in tqdm(train_df.groupby(['영업장명_메뉴명']), desc ='Training LSTM'):
        store_train = group.sort_values('영업일자').copy()
        if len(store_train) < LOOKBACK + PREDICT:
            continue

        features = ['매출수량']
        scaler = MinMaxScaler()
        store_train[features] = scaler.fit_transform(store_train[features])
        train_vals = store_train[features].values  # shape: (N, 1)

        # 시퀀스 구성
        X_train, y_train = [], []
        for i in range(len(train_vals) - LOOKBACK - PREDICT + 1):
            X_train.append(train_vals[i:i+LOOKBACK])
            y_train.append(train_vals[i+LOOKBACK:i+LOOKBACK+PREDICT, 0])

        # 리스트 → numpy → tensor 순서로 변환 (여기만 수정)
        X_train = np.array(X_train)                # (배치, time, in_dim)
        y_train = np.array(y_train)                # (배치, 예측길이)
        X_train = torch.tensor(X_train).float().to(DEVICE)
        y_train = torch.tensor(y_train).float().to(DEVICE)

        model = MultiOutputLSTM(input_dim=1, output_dim=PREDICT).to(DEVICE)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        criterion = nn.MSELoss()

        model.train()
        for epoch in range(EPOCHS):
            idx = torch.randperm(len(X_train))
            for i in range(0, len(X_train), BATCH_SIZE):
                batch_idx = idx[i:i+BATCH_SIZE]
                X_batch, y_batch = X_train[batch_idx], y_train[batch_idx]
                output = model(X_batch)
                loss = criterion(output, y_batch)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

        trained_models[store_menu] = {
            'model': model.eval(),
            'scaler': scaler,
            'last_sequence': train_vals[-LOOKBACK:]  # (28, 1)
        }

    return trained_models

In [ ]:
# 학습
trained_models = train_lstm(train)

Training LSTM: 100%|██████████| 193/193 [13:02<00:00,  4.06s/it]


# Prediction

In [ ]:
def predict_lstm(test_df, trained_models, test_prefix: str):
    results = []

    for store_menu, store_test in test_df.groupby(['영업장명_메뉴명']):
        key = store_menu
        if key not in trained_models:
            continue

        model = trained_models[key]['model']
        scaler = trained_models[key]['scaler']

        store_test_sorted = store_test.sort_values('영업일자')
        recent_vals = store_test_sorted['매출수량'].values[-LOOKBACK:]
        if len(recent_vals) < LOOKBACK:
            continue

        # 정규화
        recent_vals = scaler.transform(recent_vals.reshape(-1, 1))
        x_input = torch.tensor([recent_vals]).float().to(DEVICE)

        with torch.no_grad():
            pred_scaled = model(x_input).squeeze().cpu().numpy()

        # 역변환
        restored = []
        for i in range(PREDICT):
            dummy = np.zeros((1, 1))
            dummy[0, 0] = pred_scaled[i]
            restored_val = scaler.inverse_transform(dummy)[0, 0]
            restored.append(max(restored_val, 0))

        # 예측일자: TEST_00+1일 ~ TEST_00+7일
        pred_dates = [f"{test_prefix}+{i+1}일" for i in range(PREDICT)]

        for d, val in zip(pred_dates, restored):
            results.append({
                '영업일자': d,
                '영업장명_메뉴명': store_menu[0] if isinstance(store_menu, tuple) else store_menu, # Ensure it's a string
                '매출수량': val
            })

    return pd.DataFrame(results)

In [ ]:
all_preds = []

# 모든 test_*.csv 순회
test_files = sorted(glob.glob('/content/drive/MyDrive/lg_aimers_2/open/test/TEST_*.csv'))

for path in test_files:
    test_df = pd.read_csv(path)

    # 파일명에서 접두어 추출 (예: TEST_00)
    filename = os.path.basename(path)
    test_prefix = re.search(r'(TEST_\d+)', filename).group(1)

    pred_df = predict_lstm(test_df, trained_models, test_prefix)
    all_preds.append(pred_df)

full_pred_df = pd.concat(all_preds, ignore_index=True)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/valida

# Submission

In [ ]:
def convert_to_submission_format(pred_df: pd.DataFrame, sample_submission: pd.DataFrame):
    # (영업일자, 메뉴) → 매출수량 딕셔너리로 변환
    pred_dict = dict(zip(
        zip(pred_df['영업일자'], pred_df['영업장명_메뉴명']),
        pred_df['매출수량']
    ))

    final_df = sample_submission.copy()

    for row_idx in final_df.index:
        date = final_df.loc[row_idx, '영업일자']
        for col in final_df.columns[1:]:  # 메뉴명들
            # Retrieve the predicted value for the date and menu
            predicted_value = pred_dict.get((date, col), 0)
            # Assign the predicted value to the corresponding cell
            final_df.loc[row_idx, col] = predicted_value

    return final_df

In [ ]:
sample_submission = pd.read_csv('/content/drive/MyDrive/lg_aimers_2/open/sample_submission.csv')
submission = convert_to_submission_format(full_pred_df, sample_submission)
submission.to_csv('baseline_submission1.csv', index=False, encoding='utf-8-sig')

/tmp/ipython-input-3043597167.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '6.261391431093216' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  final_df.loc[row_idx, col] = predicted_value
/tmp/ipython-input-3043597167.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '5.913193434476852' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  final_df.loc[row_idx, col] = predicted_value
/tmp/ipython-input-3043597167.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '3.848296135663986' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  final_df.loc[row_idx, col] = predicted_value
/tmp/ipython-input-3043597167.py:16: Future

In [ ]:
submission_df = pd.read_csv('baseline_submission1.csv')
display(submission_df)

,영업일자,느티나무 셀프BBQ_1인 수저세트,느티나무 셀프BBQ_BBQ55(단체),"느티나무 셀프BBQ_대여료 30,000원","느티나무 셀프BBQ_대여료 60,000원","느티나무 셀프BBQ_대여료 90,000원","느티나무 셀프BBQ_본삼겹 (단품,실내)",느티나무 셀프BBQ_스프라이트 (단체),느티나무 셀프BBQ_신라면,느티나무 셀프BBQ_쌈야채세트,...,화담숲주막_스프라이트,화담숲주막_참살이 막걸리,화담숲주막_찹쌀식혜,화담숲주막_콜라,화담숲주막_해물파전,화담숲카페_메밀미숫가루,화담숲카페_아메리카노 HOT,화담숲카페_아메리카노 ICE,화담숲카페_카페라떼 ICE,화담숲카페_현미뻥스크림
0,TEST_00+1일,6.261391,0.000000,5.913193,3.848296,0.440351,1.307474,0.551375,2.602248,2.340865,...,4.565805,19.555780,20.383154,6.619486,59.511079,39.485751,3.354582,40.640415,9.018392,27.936639
1,TEST_00+2일,3.037337,23.374094,1.613368,0.284649,0.000000,1.284661,3.723849,3.165071,1.117158,...,0.000000,9.350769,12.785706,2.569442,31.252484,21.749821,3.304444,24.034483,7.316219,11.568531
2,TEST_00+3일,1.221648,33.658120,3.090594,0.854096,0.142106,0.883382,3.213258,2.783747,1.392788,...,0.000000,2.488278,8.038107,0.125560,7.573381,18.822619,1.260530,19.069331,5.505944,4.123553
3,TEST_00+4일,2.489965,62.895767,2.302751,1.445349,0.192107,0.941856,12.177064,4.703382,2.242055,...,0.000000,0.990587,10.556490,3.337081,9.842000,17.921908,2.024168,13.619080,6.560914,4.666720
4,TEST_00+5일,6.167986,119.342289,3.439121,1.267904,0.408124,1.173494,17.936079,7.411267,4.385892,...,5.083709,8.532410,16.494400,7.391058,37.418663,18.952586,2.013863,24.799363,6.037259,11.854491
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,TEST_09+3일,1.560406,52.354232,2.539433,0.759880,0.003157,1.295399,6.902539,0.320146,1.357876,...,2.774605,7.626598,8.639183,3.209210,53.733725,28.790556,51.194222,73.020885,16.308824,16.101036
66,TEST_09+4일,0.565045,76.143503,2.108276,1.468159,0.250592,0.824571,14.047318,0.455846,1.339219,...,0.378816,6.674483,5.776528,2.462013,39.715143,23.666706,77.579822,66.270669,17.234531,15.135924
67,TEST_09+5일,2.046006,54.047102,4.280546,1.080155,0.117614,0.912240,22.367458,0.914375,2.946263,...,0.702769,11.558890,8.849656,4.034952,54.547270,22.527404,81.886039,68.432376,14.964551,24.137706
68,TEST_09+6일,5.219364,6.258869,9.419253,3.041622,0.296617,0.939625,13.828617,2.776212,4.304093,...,2.925363,18.409632,14.347375,5.291011,97.644027,28.478755,81.960890,73.934511,14.597957,24.125728


In [ ]:
display(full_pred_df.head())

,영업일자,영업장명_메뉴명,매출수량
0,TEST_00+1일,"(느티나무 셀프BBQ_1인 수저세트,)",6.261391
1,TEST_00+2일,"(느티나무 셀프BBQ_1인 수저세트,)",3.037337
2,TEST_00+3일,"(느티나무 셀프BBQ_1인 수저세트,)",1.221648
3,TEST_00+4일,"(느티나무 셀프BBQ_1인 수저세트,)",2.489965
4,TEST_00+5일,"(느티나무 셀프BBQ_1인 수저세트,)",6.167986


In [ ]:
# Re-create pred_dict to inspect its keys
pred_dict_check = dict(zip(
    zip(full_pred_df['영업일자'], full_pred_df['영업장명_메뉴명']),
    full_pred_df['매출수량']
))

# Display some keys from the dictionary and some column names from the sample submission
print("Sample keys from pred_dict:")
print(list(pred_dict_check.keys())[:10])

print("\nSample column names from sample_submission:")
print(sample_submission.columns[1:11].tolist())

Sample keys from pred_dict:
[('TEST_00+1일', ('느티나무 셀프BBQ_1인 수저세트',)), ('TEST_00+2일', ('느티나무 셀프BBQ_1인 수저세트',)), ('TEST_00+3일', ('느티나무 셀프BBQ_1인 수저세트',)), ('TEST_00+4일', ('느티나무 셀프BBQ_1인 수저세트',)), ('TEST_00+5일', ('느티나무 셀프BBQ_1인 수저세트',)), ('TEST_00+6일', ('느티나무 셀프BBQ_1인 수저세트',)), ('TEST_00+7일', ('느티나무 셀프BBQ_1인 수저세트',)), ('TEST_00+1일', ('느티나무 셀프BBQ_BBQ55(단체)',)), ('TEST_00+2일', ('느티나무 셀프BBQ_BBQ55(단체)',)), ('TEST_00+3일', ('느티나무 셀프BBQ_BBQ55(단체)',))]

Sample column names from sample_submission:
['느티나무 셀프BBQ_1인 수저세트', '느티나무 셀프BBQ_BBQ55(단체)', '느티나무 셀프BBQ_대여료 30,000원', '느티나무 셀프BBQ_대여료 60,000원', '느티나무 셀프BBQ_대여료 90,000원', '느티나무 셀프BBQ_본삼겹 (단품,실내)', '느티나무 셀프BBQ_스프라이트 (단체)', '느티나무 셀프BBQ_신라면', '느티나무 셀프BBQ_쌈야채세트', '느티나무 셀프BBQ_쌈장']


In [ ]:
from google.colab import files
files.download('baseline_submission1.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

To use a GPU in Google Colab, go to `Runtime` -> `Change runtime type` and select `GPU` under `Hardware accelerator`.